In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import umap

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import umap
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.image as img
from matplotlib.image import imread

from PIL import Image,ImageOps

from sklearn.decomposition import PCA

from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

import torch

In [ ]:
import matplotlib.pyplot as plt 
plt.rcParams.update({
    "font.family" : 'Arial',   # specify font family here
    "font.size" : 12,
    "font.weight" : "normal"})          # specify font size here

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
base_path = '/home/jinny/projects/Art-history/Art-history/datas/'
file_info = pd.read_csv(base_path+'file_info.csv')

# file_info_latent
df = pd.DataFrame(( np.load(base_path+'vectors/avec_latents.npy', allow_pickle=True)),columns=['avec','Path'])
file_info_latents = pd.merge(file_info, df, how = 'left', on = 'Path')
file_info_latents = file_info_latents[~file_info_latents.avec.isnull()]

avec = np.array([i.reshape(-1) for i in file_info_latents['avec']])

In [ ]:
df = pd.DataFrame(np.load(base_path+'vectors/cvec_latents.npy', allow_pickle=True),columns=['cvec','Path'])
file_info_latents = pd.merge(file_info_latents, df, how = 'left', on = 'Path')

cvec = np.array([i.reshape(-1) for i in file_info_latents['cvec']])

In [ ]:
pca_avec = PCA(n_components=100,random_state=42)
pca_fit = pca_avec.fit_transform(avec)
pca_avec_df = pd.DataFrame(data=pca_fit)

pca_avec_df.columns = ['avec_pc'+str(i+1) for i in range(100)]

In [ ]:
pca_cvec = PCA(n_components=100,random_state=42)
pca_fit = pca_cvec.fit_transform(cvec)
pca_cvec_df = pd.DataFrame(data=pca_fit)

pca_cvec_df.columns = ['cvec_pc'+str(i+1) for i in range(100)]

In [ ]:
file_info_latents = pd.concat([file_info_latents,pca_avec_df.iloc[:,:10]],axis=1)
file_info_latents = pd.concat([file_info_latents,pca_cvec_df.iloc[:,:10]],axis=1)

In [ ]:
temp_file_info_latents = file_info_latents[file_info_latents['new_date']>=1500].reset_index(drop=True)

### pca value

In [ ]:
a_eigen_variance = pca_avec.explained_variance_ratio_
c_eigen_variance = pca_cvec.explained_variance_ratio_

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(9,7))
plt.scatter(np.arange(1,len(a_eigen_variance)+1),a_eigen_variance,s=10)
plt.yscale('log')
plt.xticks(fontsize=28)
plt.yticks(fontsize=28)
plt.title('A-vector eigen value',fontsize=34)
plt.savefig(
    f"/home/jinny/projects/Art-history/Art-history/graph/supple/si09_avec.svg",
    bbox_inches="tight",
    transparent=True,
    dpi=600,
)
plt.show()

In [ ]:
a_eigen_variance[:10]

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(9,7))
plt.scatter(np.arange(1,len(c_eigen_variance)+1),c_eigen_variance,s=10)
plt.yscale('log')
plt.xticks(fontsize=28)
plt.yticks(fontsize=28)
plt.title('C-vector eigen value',fontsize=34)
plt.savefig(
    f"/home/jinny/projects/Art-history/Art-history/graph/supple/si09_cvec.svg",
    bbox_inches="tight",
    transparent=True,
    dpi=600,
)
plt.show()

In [ ]:
c_eigen_variance[:10]

### pca pc1 - pc10

In [ ]:
def make_pca_ridge(pc, vec, ticks):
    sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

    # new_date 값을 내림차순으로 정렬
    temp_file_info_latents_sorted = temp_file_info_latents.sort_values(by="new_date", ascending=False)
    unique_dates = sorted(temp_file_info_latents["new_date"].unique())

    # FacetGrid를 사용하여 데이터 분리 (내림차순 정렬된 데이터 사용)
    g = sns.FacetGrid(
        temp_file_info_latents,
        row="new_date",
        hue="new_date",
        aspect=30,
        height=0.2,
        palette="Spectral_r",
        row_order=unique_dates,
    )
    years = list(temp_file_info_latents["new_date"].unique())

    # 각 능선에 KDE 플롯 추가
    g.map(sns.kdeplot, f"{vec}vec_pc{pc}", clip_on=False, shade=True, alpha=1, lw=1, bw_adjust=0.5)
    g.map(sns.kdeplot, f"{vec}vec_pc{pc}", clip_on=False, color="w", lw=1.5, bw_adjust=0.5)

    # 각 능선의 오버랩을 조정
    g.map(plt.axhline, y=0, lw=2, clip_on=False)

    # 그래프의 각 요소를 디자인
    g.fig.subplots_adjust(hspace=-0.6)

    # 범례 제거 및 기본 설정
    g.set_titles("")
    g.set(yticks=[], ylabel=None, xlabel=None)
    g.despine(bottom=True, left=True)

    # # x축 수치 표시 - 마지막 subplot에만 적용
    # for ax in g.axes[:-1].flat:
    #     ax.set_xticks([])
    
    bottom_ax = g.axes[-1, 0]
    bottom_ax.set_xticks(ticks,fontsize=25)  # x축 눈금을 설정
    bottom_ax.set_xticklabels(ticks, fontsize=25)


    # Colorbar 추가
    norm = Normalize(
        vmin=min([years.index(x) for x in temp_file_info_latents["new_date"]]),
        vmax=max([years.index(x) for x in temp_file_info_latents["new_date"]]),
    )
    cbar_ax = g.fig.add_axes([0.95, 0.1, 0.03, 0.75])
    sm = ScalarMappable(cmap="Spectral_r", norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(
        sm, ax=g.axes.ravel().tolist(), cax=cbar_ax, orientation="vertical", aspect=30, fraction=0.05
    )
    cbar.set_ticks([0, 25, 49])
    cbar.set_ticklabels([1500, 1750, 1990], fontsize=36)
    cbar.ax.invert_yaxis()

    # 제목 추가
    g.fig.suptitle(f"PC{pc}", fontsize=46, y=0.95, x=0.6)  # y로 타이틀 위치 조정
    plt.savefig(
        f"/home/jinny/projects/Art-history/Art-history/graph/supple/si10_{vec}vec_pc{pc}.png",
        bbox_inches="tight",
        transparent=True,
        dpi=600,
    )
    plt.show()


In [ ]:
num = 3
print(pca_cvec_df[f'cvec_pc{num}'].min())
print(pca_cvec_df[f'cvec_pc{num}'].max())

In [ ]:
make_pca_ridge(3,'a',range(-110,111,55))
make_pca_ridge(4,'a',range(-100,101,50))
make_pca_ridge(5,'a',range(-120,121,60))
make_pca_ridge(6,'a',range(-90,91,45))
make_pca_ridge(7,'a',range(-120,121,60))
make_pca_ridge(8,'a',range(-80,81,40))
make_pca_ridge(9,'a',range(-80,81,40))
make_pca_ridge(10,'a',range(-60,61,30))

In [ ]:
make_pca_ridge(3,'c',np.arange(-0.5,0.51,0.25))
make_pca_ridge(4,'c',np.arange(-0.5,0.51,0.25))
make_pca_ridge(5,'c',np.arange(-0.5,0.51,0.25))
make_pca_ridge(6,'c',np.arange(-0.5,0.51,0.25))
make_pca_ridge(7,'c',np.arange(-0.5,0.51,0.25))
make_pca_ridge(8,'c',np.arange(-0.5,0.51,0.25))
make_pca_ridge(9,'c',np.arange(-0.5,0.51,0.25))
make_pca_ridge(10,'c',np.arange(-0.5,0.51,0.25))